<a href="https://colab.research.google.com/github/Jeevan-hub1/AI-POWERED-DRUG-DISCOVERY-AND-DEVELOPMENT-SYSTEM/blob/main/drug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# AI-Powered Drug Discovery and Development
# Fixed Novel Deep Learning Algorithm with Cross-Validation and Real-time Visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Deep Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, GRU
from tensorflow.keras.layers import Embedding, Flatten, MultiHeadAttention, Reshape
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Utilities
import pickle
import joblib
import os
import time
from datetime import datetime
import json

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class DrugDiscoveryAI:
    """
    Fixed AI-Powered Drug Discovery Algorithm

    This class implements a hybrid deep learning approach combining:
    1. Molecular fingerprint analysis
    2. Multi-modal neural networks
    3. Attention mechanisms
    4. Ensemble learning
    """

    def __init__(self, config=None):
        """Initialize the DrugDiscoveryAI system"""
        self.config = config or self._default_config()
        self.models = {}
        self.scalers = {}
        self.encoders = {}
        self.feature_importance = {}
        self.training_history = {}
        self.cross_validation_results = {}

    def _default_config(self):
        """Default configuration for the system"""
        return {
            'molecular_fingerprint_size': 2048,
            'embedding_dim': 128,
            'lstm_units': 256,
            'attention_heads': 8,
            'dense_units': [512, 256, 128],
            'dropout_rate': 0.3,
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 20,
            'patience': 15,
            'cv_folds': 5
        }

    def load_and_preprocess_data(self, data_path=None):
        """
        Load and preprocess drug discovery datasets
        Uses synthetic realistic drug discovery data
        """
        print("Loading and preprocessing drug discovery data...")

        # Generate synthetic realistic drug discovery data
        np.random.seed(42)

        # Generate molecular data
        n_samples = 10000

        # Molecular descriptors (based on common drug properties)
        molecular_weight = np.random.normal(300, 100, n_samples)
        logp = np.random.normal(2.5, 1.2, n_samples)
        hbd = np.random.poisson(2, n_samples)  # Hydrogen bond donors
        hba = np.random.poisson(4, n_samples)  # Hydrogen bond acceptors
        tpsa = np.random.normal(60, 30, n_samples)  # Topological polar surface area
        rotatable_bonds = np.random.poisson(5, n_samples)

        # Generate SMILES-like fingerprints
        fingerprints = np.random.randint(0, 2, (n_samples, self.config['molecular_fingerprint_size']))

        # Target variables (drug efficacy, toxicity, solubility)
        efficacy = np.random.beta(2, 2, n_samples)  # 0-1 scale
        toxicity = np.random.beta(1, 3, n_samples)  # Lower is better
        solubility = np.random.normal(0, 2, n_samples)  # Log scale

        # Binary classification: Drug-like vs Non-drug-like
        drug_like = ((molecular_weight < 500) &
                    (logp < 5) &
                    (hbd <= 5) &
                    (hba <= 10) &
                    (tpsa < 140)).astype(int)

        # Create comprehensive dataset
        self.data = pd.DataFrame({
            'molecular_weight': molecular_weight,
            'logp': logp,
            'hbd': hbd,
            'hba': hba,
            'tpsa': tpsa,
            'rotatable_bonds': rotatable_bonds,
            'efficacy': efficacy,
            'toxicity': toxicity,
            'solubility': solubility,
            'drug_like': drug_like
        })

        # Add fingerprint data
        fingerprint_df = pd.DataFrame(fingerprints,
                                    columns=[f'fp_{i}' for i in range(self.config['molecular_fingerprint_size'])])
        self.data = pd.concat([self.data, fingerprint_df], axis=1)

        # Add compound IDs
        self.data['compound_id'] = [f'COMP_{i:06d}' for i in range(n_samples)]

        print(f"Dataset loaded: {len(self.data)} compounds")
        print(f"Features: {self.data.shape[1]}")
        print(f"Drug-like compounds: {self.data['drug_like'].sum()}")

        return self.data

    def extract_molecular_features(self, smiles_list=None):
        """
        Extract molecular features from SMILES strings
        """
        print("Extracting molecular features...")

        molecular_features = self.data[['molecular_weight', 'logp', 'hbd', 'hba', 'tpsa', 'rotatable_bonds']]
        fingerprint_features = self.data[[col for col in self.data.columns if col.startswith('fp_')]]

        return molecular_features, fingerprint_features

    def build_novel_architecture(self, input_shape_mol, input_shape_fp):
        """
        Build novel hybrid deep learning architecture - FIXED VERSION

        Architecture combines:
        1. Molecular descriptor processing branch
        2. Fingerprint processing branch with attention
        3. Multi-task learning for multiple outputs
        """
        print("Building novel hybrid architecture...")

        # Molecular descriptor branch
        molecular_input = Input(shape=(input_shape_mol,), name='molecular_input')
        mol_dense1 = Dense(256, activation='relu')(molecular_input)
        mol_bn1 = BatchNormalization()(mol_dense1)
        mol_dropout1 = Dropout(self.config['dropout_rate'])(mol_bn1)

        mol_dense2 = Dense(128, activation='relu')(mol_dropout1)
        mol_bn2 = BatchNormalization()(mol_dense2)
        mol_dropout2 = Dropout(self.config['dropout_rate'])(mol_bn2)

        # Fingerprint branch with attention
        fingerprint_input = Input(shape=(input_shape_fp,), name='fingerprint_input')

        # Reshape for 1D convolution - FIXED
        fp_reshaped = Reshape((input_shape_fp, 1))(fingerprint_input)

        # Convolutional layers for pattern recognition
        conv1 = Conv1D(64, 3, activation='relu', padding='same')(fp_reshaped)
        conv1 = MaxPooling1D(2)(conv1)
        conv1 = BatchNormalization()(conv1)

        conv2 = Conv1D(128, 3, activation='relu', padding='same')(conv1)
        conv2 = MaxPooling1D(2)(conv2)
        conv2 = BatchNormalization()(conv2)

        # LSTM for sequence modeling
        lstm_out = LSTM(self.config['lstm_units'], return_sequences=True)(conv2)
        lstm_out = Dropout(self.config['dropout_rate'])(lstm_out)

        # Self-attention mechanism - FIXED
        attention_out = MultiHeadAttention(
            num_heads=self.config['attention_heads'],
            key_dim=self.config['lstm_units'] // self.config['attention_heads']
        )(lstm_out, lstm_out)

        # Global pooling
        fp_global = GlobalMaxPooling1D()(attention_out)

        # Fusion layer
        fused = Concatenate()([mol_dropout2, fp_global])

        # Shared dense layers
        shared_dense1 = Dense(512, activation='relu')(fused)
        shared_bn1 = BatchNormalization()(shared_dense1)
        shared_dropout1 = Dropout(self.config['dropout_rate'])(shared_bn1)

        shared_dense2 = Dense(256, activation='relu')(shared_dropout1)
        shared_bn2 = BatchNormalization()(shared_dense2)
        shared_dropout2 = Dropout(self.config['dropout_rate'])(shared_bn2)

        # Multi-task outputs - FIXED
        drug_like_output = Dense(1, activation='sigmoid', name='drug_like')(shared_dropout2)
        efficacy_output = Dense(1, activation='sigmoid', name='efficacy')(shared_dropout2)
        toxicity_output = Dense(1, activation='sigmoid', name='toxicity')(shared_dropout2)
        solubility_output = Dense(1, activation='linear', name='solubility')(shared_dropout2)

        # Create model
        model = Model(
            inputs=[molecular_input, fingerprint_input],
            outputs=[drug_like_output, efficacy_output, toxicity_output, solubility_output]
        )

        return model

    def train_model(self, X_mol, X_fp, y_drug_like, y_efficacy, y_toxicity, y_solubility):
        """
        Train the novel hybrid model with cross-validation
        """
        print("Training hybrid deep learning model...")

        # Scale features
        self.scalers['molecular'] = StandardScaler()
        self.scalers['fingerprint'] = StandardScaler()

        X_mol_scaled = self.scalers['molecular'].fit_transform(X_mol)
        X_fp_scaled = self.scalers['fingerprint'].fit_transform(X_fp)

        # Split data
        X_mol_train, X_mol_test, X_fp_train, X_fp_test, y_drug_train, y_drug_test, \
        y_eff_train, y_eff_test, y_tox_train, y_tox_test, y_sol_train, y_sol_test = train_test_split(
            X_mol_scaled, X_fp_scaled, y_drug_like, y_efficacy, y_toxicity, y_solubility,
            test_size=0.2, random_state=42, stratify=y_drug_like
        )

        # Build model
        model = self.build_novel_architecture(X_mol_train.shape[1], X_fp_train.shape[1])

        # Compile with multi-task loss
        model.compile(
            optimizer=Adam(learning_rate=self.config['learning_rate']),
            loss={
                'drug_like': 'binary_crossentropy',
                'efficacy': 'mse',
                'toxicity': 'mse',
                'solubility': 'mse'
            },
            loss_weights={
                'drug_like': 1.0,
                'efficacy': 0.5,
                'toxicity': 0.5,
                'solubility': 0.3
            },
            metrics={
                'drug_like': ['accuracy'],
                'efficacy': ['mae'],
                'toxicity': ['mae'],
                'solubility': ['mae']
            }
        )

        # Callbacks
        callbacks = [
            EarlyStopping(patience=self.config['patience'], restore_best_weights=True),
            ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
        ]

        # Train model
        history = model.fit(
            [X_mol_train, X_fp_train],
            {
                'drug_like': y_drug_train,
                'efficacy': y_eff_train,
                'toxicity': y_tox_train,
                'solubility': y_sol_train
            },
            validation_data=(
                [X_mol_test, X_fp_test],
                {
                    'drug_like': y_drug_test,
                    'efficacy': y_eff_test,
                    'toxicity': y_tox_test,
                    'solubility': y_sol_test
                }
            ),
            epochs=self.config['epochs'],
            batch_size=self.config['batch_size'],
            callbacks=callbacks,
            verbose=1
        )

        self.models['hybrid'] = model
        self.training_history = history.history

        # Evaluate model
        test_loss = model.evaluate(
            [X_mol_test, X_fp_test],
            {
                'drug_like': y_drug_test,
                'efficacy': y_eff_test,
                'toxicity': y_tox_test,
                'solubility': y_sol_test
            },
            verbose=0
        )

        print(f"Test Loss: {test_loss[0]:.4f}")

        return model, history

    def cross_validate_model(self, X_mol, X_fp, y_drug_like, y_efficacy, y_toxicity, y_solubility):
        """
        Perform cross-validation to ensure robustness
        """
        print("Performing cross-validation...")

        kfold = StratifiedKFold(n_splits=self.config['cv_folds'], shuffle=True, random_state=42)
        cv_scores = {
            'drug_like_acc': [],
            'efficacy_mae': [],
            'toxicity_mae': [],
            'solubility_mae': []
        }

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X_mol, y_drug_like)):
            print(f"Training fold {fold + 1}/{self.config['cv_folds']}")

            # Split data
            X_mol_train_fold = self.scalers['molecular'].fit_transform(X_mol[train_idx])
            X_mol_val_fold = self.scalers['molecular'].transform(X_mol[val_idx])
            X_fp_train_fold = self.scalers['fingerprint'].fit_transform(X_fp[train_idx])
            X_fp_val_fold = self.scalers['fingerprint'].transform(X_fp[val_idx])

            y_drug_train_fold = y_drug_like[train_idx]
            y_drug_val_fold = y_drug_like[val_idx]
            y_eff_train_fold = y_efficacy[train_idx]
            y_eff_val_fold = y_efficacy[val_idx]
            y_tox_train_fold = y_toxicity[train_idx]
            y_tox_val_fold = y_toxicity[val_idx]
            y_sol_train_fold = y_solubility[train_idx]
            y_sol_val_fold = y_solubility[val_idx]

            # Build and train model
            model = self.build_novel_architecture(X_mol_train_fold.shape[1], X_fp_train_fold.shape[1])
            model.compile(
                optimizer=Adam(learning_rate=self.config['learning_rate']),
                loss={
                    'drug_like': 'binary_crossentropy',
                    'efficacy': 'mse',
                    'toxicity': 'mse',
                    'solubility': 'mse'
                },
                loss_weights={
                    'drug_like': 1.0,
                    'efficacy': 0.5,
                    'toxicity': 0.5,
                    'solubility': 0.3
                },
                metrics={
                    'drug_like': ['accuracy'],
                    'efficacy': ['mae'],
                    'toxicity': ['mae'],
                    'solubility': ['mae']
                }
            )

            # Train with reduced epochs for CV
            model.fit(
                [X_mol_train_fold, X_fp_train_fold],
                {
                    'drug_like': y_drug_train_fold,
                    'efficacy': y_eff_train_fold,
                    'toxicity': y_tox_train_fold,
                    'solubility': y_sol_train_fold
                },
                epochs=20,  # Reduced for CV
                batch_size=self.config['batch_size'],
                verbose=0
            )

            # Evaluate
            predictions = model.predict([X_mol_val_fold, X_fp_val_fold])

            # Calculate metrics
            drug_like_acc = np.mean((predictions[0] > 0.5).astype(int) == y_drug_val_fold)
            efficacy_mae = np.mean(np.abs(predictions[1].flatten() - y_eff_val_fold))
            toxicity_mae = np.mean(np.abs(predictions[2].flatten() - y_tox_val_fold))
            solubility_mae = np.mean(np.abs(predictions[3].flatten() - y_sol_val_fold))

            cv_scores['drug_like_acc'].append(drug_like_acc)
            cv_scores['efficacy_mae'].append(efficacy_mae)
            cv_scores['toxicity_mae'].append(toxicity_mae)
            cv_scores['solubility_mae'].append(solubility_mae)

        # Calculate mean and std
        self.cross_validation_results = {
            'drug_like_acc': {
                'mean': np.mean(cv_scores['drug_like_acc']),
                'std': np.std(cv_scores['drug_like_acc'])
            },
            'efficacy_mae': {
                'mean': np.mean(cv_scores['efficacy_mae']),
                'std': np.std(cv_scores['efficacy_mae'])
            },
            'toxicity_mae': {
                'mean': np.mean(cv_scores['toxicity_mae']),
                'std': np.std(cv_scores['toxicity_mae'])
            },
            'solubility_mae': {
                'mean': np.mean(cv_scores['solubility_mae']),
                'std': np.std(cv_scores['solubility_mae'])
            }
        }

        print("Cross-validation results:")
        for metric, results in self.cross_validation_results.items():
            print(f"{metric}: {results['mean']:.4f} ± {results['std']:.4f}")

        return self.cross_validation_results

    def create_dashboard(self):
        """
        Create real-time advanced data visualization dashboard
        """
        print("Creating advanced visualization dashboard...")

        # 1. Data Overview Dashboard
        fig_overview = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Molecular Weight Distribution', 'LogP vs Drug-likeness',
                          'Efficacy vs Toxicity', 'Feature Correlation Heatmap'),
            specs=[[{'type': 'histogram'}, {'type': 'scatter'}],
                   [{'type': 'scatter'}, {'type': 'heatmap'}]]
        )

        # Molecular weight distribution
        fig_overview.add_trace(
            go.Histogram(x=self.data['molecular_weight'], name='Molecular Weight',
                        nbinsx=50, marker_color='skyblue'),
            row=1, col=1
        )

        # LogP vs Drug-likeness
        colors = ['red' if x == 0 else 'green' for x in self.data['drug_like']]
        fig_overview.add_trace(
            go.Scatter(x=self.data['logp'], y=self.data['molecular_weight'],
                      mode='markers', marker=dict(color=colors, size=5),
                      name='Drug-like compounds'),
            row=1, col=2
        )

        # Efficacy vs Toxicity
        fig_overview.add_trace(
            go.Scatter(x=self.data['efficacy'], y=self.data['toxicity'],
                      mode='markers', marker=dict(color='purple', size=5),
                      name='Efficacy vs Toxicity'),
            row=2, col=1
        )

        # Feature correlation heatmap
        corr_features = ['molecular_weight', 'logp', 'hbd', 'hba', 'tpsa', 'rotatable_bonds']
        corr_matrix = self.data[corr_features].corr()

        fig_overview.add_trace(
            go.Heatmap(z=corr_matrix.values, x=corr_features, y=corr_features,
                      colorscale='RdBu', zmid=0),
            row=2, col=2
        )

        fig_overview.update_layout(
            title_text="Drug Discovery Data Overview Dashboard",
            showlegend=True,
            height=800
        )

        return fig_overview

    def predict_drug_properties(self, compounds):
        """
        Predict drug properties for new compounds
        """
        if 'hybrid' not in self.models:
            print("Model not trained yet. Please train the model first.")
            return None

        # Preprocess compounds
        X_mol_scaled = self.scalers['molecular'].transform(compounds['molecular'])
        X_fp_scaled = self.scalers['fingerprint'].transform(compounds['fingerprint'])

        # Make predictions
        predictions = self.models['hybrid'].predict([X_mol_scaled, X_fp_scaled])

        results = {
            'drug_like_probability': predictions[0].flatten(),
            'efficacy_prediction': predictions[1].flatten(),
            'toxicity_prediction': predictions[2].flatten(),
            'solubility_prediction': predictions[3].flatten()
        }

        return results

    def save_model(self, filepath):
        """Save the trained model and preprocessing objects"""
        try:
            # Save model
            if 'hybrid' in self.models:
                self.models['hybrid'].save(f"{filepath}_model.h5")

            # Save preprocessing objects
            joblib.dump(self.scalers, f"{filepath}_scalers.pkl")
            joblib.dump(self.cross_validation_results, f"{filepath}_cv_results.pkl")

            print(f"Model saved to {filepath}")
        except Exception as e:
            print(f"Error saving model: {e}")

    def load_model(self, filepath):
        """Load a trained model and preprocessing objects"""
        try:
            # Load model
            self.models['hybrid'] = tf.keras.models.load_model(f"{filepath}_model.h5")

            # Load preprocessing objects
            self.scalers = joblib.load(f"{filepath}_scalers.pkl")
            self.cross_validation_results = joblib.load(f"{filepath}_cv_results.pkl")

            print(f"Model loaded from {filepath}")
        except Exception as e:
            print(f"Error loading model: {e}")

# Main execution
def main():
    """
    Main function to run the AI-Powered Drug Discovery system
    """
    print("="*60)
    print("AI-POWERED DRUG DISCOVERY AND DEVELOPMENT SYSTEM")
    print("="*60)

    # Initialize system
    ai_system = DrugDiscoveryAI()

    # Load and preprocess data
    data = ai_system.load_and_preprocess_data()

    # Extract features
    molecular_features, fingerprint_features = ai_system.extract_molecular_features()

    # Prepare targets
    y_drug_like = data['drug_like'].values
    y_efficacy = data['efficacy'].values
    y_toxicity = data['toxicity'].values
    y_solubility = data['solubility'].values

    # Train model
    print("\n" + "="*40)
    print("TRAINING PHASE")
    print("="*40)

    model, history = ai_system.train_model(
        molecular_features.values, fingerprint_features.values,
        y_drug_like, y_efficacy, y_toxicity, y_solubility
    )

    # Cross-validation
    print("\n" + "="*40)
    print("CROSS-VALIDATION PHASE")
    print("="*40)

    cv_results = ai_system.cross_validate_model(
        molecular_features.values, fingerprint_features.values,
        y_drug_like, y_efficacy, y_toxicity, y_solubility
    )

    # Create visualization dashboard
    print("\n" + "="*40)
    print("CREATING VISUALIZATION DASHBOARD")
    print("="*40)

    dashboard = ai_system.create_dashboard()

    # Generate detailed results report
    print("\n" + "="*40)
    print("FINAL RESULTS SUMMARY")
    print("="*40)

    print(f"Dataset Size: {len(data)} compounds")
    print(f"Drug-like Compounds: {data['drug_like'].sum()} ({data['drug_like'].mean()*100:.1f}%)")
    print(f"Average Molecular Weight: {data['molecular_weight'].mean():.2f} Da")
    print(f"Average LogP: {data['logp'].mean():.2f}")

    print("\nCross-Validation Results:")
    for metric, results in cv_results.items():
        print(f"  {metric}: {results['mean']:.4f} ± {results['std']:.4f}")

    # Save model
    ai_system.save_model("drug_discovery_model")

    # Example predictions on new compounds
    print("\n" + "="*40)
    print("EXAMPLE PREDICTIONS")
    print("="*40)

    # Take first 5 compounds as example
    example_compounds = {
        'molecular': molecular_features.iloc[:5].values,
        'fingerprint': fingerprint_features.iloc[:5].values
    }

    predictions = ai_system.predict_drug_properties(example_compounds)

    if predictions:
        print("Sample Predictions:")
        for i in range(5):
            print(f"Compound {i+1}:")
            print(f"  Drug-like probability: {predictions['drug_like_probability'][i]:.3f}")
            print(f"  Efficacy prediction: {predictions['efficacy_prediction'][i]:.3f}")
            print(f"  Toxicity prediction: {predictions['toxicity_prediction'][i]:.3f}")
            print(f"  Solubility prediction: {predictions['solubility_prediction'][i]:.3f}")
            print()

    print("="*60)
    print("DRUG DISCOVERY ANALYSIS COMPLETE")
    print("="*60)

    return ai_system

# Execute the main function
if __name__ == "__main__":
    try:
        # Run main analysis
        system = main()

        # Generate final report
        final_report = {
            'dataset_size': len(system.data),
            'drug_like_count': int(system.data['drug_like'].sum()),
            'cv_results': system.cross_validation_results,
            'timestamp': datetime.now().isoformat()
        }

        # Save final results
        with open('drug_discovery_report.json', 'w') as f:
            json.dump(final_report, f, indent=2, default=str)

        print("\nAll results saved successfully!")
        print("Files generated:")
        print("- drug_discovery_model_model.h5")
        print("- drug_discovery_model_scalers.pkl")
        print("- drug_discovery_model_cv_results.pkl")
        print("- drug_discovery_report.json")

    except Exception as e:
        print(f"Error during execution: {str(e)}")
        import traceback
        traceback.print_exc()
        print("Please check the requirements and data availability.")

AI-POWERED DRUG DISCOVERY AND DEVELOPMENT SYSTEM
Loading and preprocessing drug discovery data...
Dataset loaded: 10000 compounds
Features: 2059
Drug-like compounds: 9358
Extracting molecular features...

TRAINING PHASE
Training hybrid deep learning model...
Building novel hybrid architecture...
Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 38s 98ms/step - drug_like_accuracy: 0.7258 - drug_like_loss: 0.5770 - efficacy_loss: 0.1168 - efficacy_mae: 0.2800 - loss: 2.4481 - solubility_loss: 5.8084 - solubility_mae: 1.9109 - toxicity_loss: 0.1403 - toxicity_mae: 0.2997 - val_drug_like_accuracy: 0.9360 - val_drug_like_loss: 0.2320 - val_efficacy_loss: 0.0493 - val_efficacy_mae: 0.1857 - val_loss: 1.4761 - val_solubility_loss: 3.9731 - val_solubility_mae: 1.6031 - val_toxicity_loss: 0.0397 - val_toxicity_mae: 0.1619 - learning_rate: 0.0010
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 24s 97ms/step - drug_like_accuracy: 0.9252 - drug_like_loss: 0.2045 - efficacy_loss: 0.0719 - efficacy_mae: 0.2189 - loss


FINAL RESULTS SUMMARY
Dataset Size: 10000 compounds
Drug-like Compounds: 9358 (93.6%)
Average Molecular Weight: 299.79 Da
Average LogP: 2.52

Cross-Validation Results:
  drug_like_acc: 0.8917 ± 0.0202
  efficacy_mae: 0.1896 ± 0.0058
  toxicity_mae: 0.1669 ± 0.0153
  solubility_mae: 1.5923 ± 0.0114
Model saved to drug_discovery_model

EXAMPLE PREDICTIONS
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
Sample Predictions:
Compound 1:
  Drug-like probability: 1.000
  Efficacy prediction: 0.523
  Toxicity prediction: 0.266
  Solubility prediction: -0.180

Compound 2:
  Drug-like probability: 0.997
  Efficacy prediction: 0.533
  Toxicity prediction: 0.257
  Solubility prediction: 0.251

Compound 3:
  Drug-like probability: 1.000
  Efficacy prediction: 0.508
  Toxicity prediction: 0.246
  Solubility prediction: -0.093

Compound 4:
  Drug-like probability: 0.999
  Efficacy prediction: 0.497
  Toxicity prediction: 0.230
  Solubility prediction: -0.277

Compound 5:
  Drug-like probability: 1.000
  Effi

SyntaxError: invalid syntax (ipython-input-3-2607504187.py, line 1)

In [29]:
!git add .config/ drug_discovery_model_cv_results.pkl drug_discovery_model_model.h5 drug_discovery_model_scalers.pkl drug_discovery_report.json sample_data/

In [43]:
!git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [45]:
!git config --global user.name "Jeevan-hub1"
!git config --global user.email "nandakumarponnala@gmail.com"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')